# CEE6501 — Coding Assignment, Week 5

**Assigned:** 02/13/2026 (Week 5)  
**Due:** 02/23/2026

**Canvas Submission Link:**  <https://gatech.instructure.com/courses/517856/assignments/2320760>

---

## Logistics

### 💻 Assignment Format

This is a **coding assignment**.

- Complete the assignment by **executing and completing all tasks in the notebook cells below**
- The notebook should be run and completed in **Google Colab**
- Your submission **must be a link to a functioning Google Colab notebook**

You may use any local tools (VS Code, JupyterLab, etc.) while working, but the **final submitted work must run correctly in Colab**.


### 📤 Submission Instructions

- Submit **one link** to your Google Colab notebook on Canvas
- Ensure that:
  - All cells run **top-to-bottom without errors**
  - All required outputs are visible
  - The notebook reflects your final answers

### ✅ Checklist Before Submitting

- [ ] All notebook cells completed
- [ ] Code runs without errors from a fresh runtime
- [ ] Outputs and plots are clearly visible
- [ ] Colab link opens and runs correctly
- [ ] Correct notebook submitted on Canvas

### 🤝 Collaboration / AI tools
You may discuss concepts with classmates and you may use AI tools to help you learn,
but **your submitted code must be written by you and you must understand it**.
If you used outside help, add a short note in the final reflection cell.

---

## --- Google Colab environment setup ---

The cell below only needs to run when the notebook is opened in Google Colab.

This code will not affect code execution locally in VS-code + conda environment.

Google Colab starts each session with its own **preloaded versions** of common Python (currently 3.12.12) and Python packages (NumPy, SciPy, etc.).  
If we install different package versions once loaded, Python cannot switch to them while it is already running.

### What will happen
When you run the setup cell below in Google Colab:

1. The required package versions are installed
2. The runtime is **automatically restarted** so the new versions can be loaded  
3. You may see the message **“Your session crashed for an unknown reason.”**  
   → This is expected and normal

After the restart, rerun the notebook and check the **version check cell** to confirm package versions are correct.

### Runtime menu notes
- **Runtime → Restart session**  
  Restarts Python but keeps installed packages and saved files

- **Runtime → Disconnect and delete runtime**  
  Resets Colab completely to its default environment (packages will need to be reinstalled)

In [52]:
# ============================================================
# Google Colab environment setup (pinned versions)
# ============================================================

import sys
import os
import subprocess

if "google.colab" in sys.modules:
    print("Running in Google Colab")
    print("Python version:", sys.version.split()[0])

    # ---- Required package versions --------------------------
    requirements = {
        "numpy": "2.4.0",
        "scipy": "1.16.3",
        "matplotlib": "3.10.8",
        "pandas": "2.3.3",
        "plotly": "6.5.2"
    }

    # ---- Check currently loaded versions --------------------
    restart_needed = False

    for pkg, required_version in requirements.items():
        try:
            module = __import__(pkg)
            installed_version = module.__version__
        except Exception:
            installed_version = None

        print(f"{pkg}: {installed_version} (required: {required_version})")

        if installed_version != required_version:
            restart_needed = True

    # ---- Install if needed ----------------------------------
    if restart_needed:
        print("\nInstalling pinned package versions...")

        pip_args = [
            f"{pkg}=={ver}" for pkg, ver in requirements.items()
        ]

        subprocess.check_call(
            [sys.executable, "-m", "pip", "install", "-q", *pip_args]
        )

        print("Installation complete.")
        print("Restarting runtime to load correct packages...")

        # This will appear as a "crash" in Colab — expected behavior
        os.kill(os.getpid(), 9)

    else:
        print("\nAll required package versions already installed.")

else:
    print("Not running in Google Colab — setup skipped.")
    print("Python version:", sys.version.split()[0])

Not running in Google Colab — setup skipped.
Python version: 3.12.12


In [53]:
# --- Version check ---
import numpy
import scipy
import matplotlib
import pandas
import plotly

print("numpy:", numpy.__version__)
print("scipy:", scipy.__version__)
print("matplotlib:", matplotlib.__version__)
print("pandas:", pandas.__version__)
print("plotly:", plotly.__version__)

numpy: 2.4.0
scipy: 1.16.3
matplotlib: 3.10.8
pandas: 2.3.3
plotly: 6.5.2


---
---

## Imports

Run this cell once before starting the assignment.

In [54]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import plotly.graph_objects as go

np.set_printoptions(precision=3, suppress=True)


---

## GIVEN CODE (You will need to modify):

## Parsing & Helper Functions

In [55]:
def restrained_dofs_1based(nodes_restrained, node_dofs_1based):
    """Return sorted list of restrained DOFs (1-based) from node restraints."""
    dof_restrained = []

    for node, restraints in nodes_restrained.items():
        ux_dof, uy_dof = node_dofs_1based(node)
        if "ux" in restraints:
            dof_restrained.append(ux_dof)
        if "uy" in restraints:
            dof_restrained.append(uy_dof)

    return sorted(dof_restrained)

In [56]:
def loaded_dofs_1based(nodes_loaded, node_dofs_1based):
    """Return DOF→load mapping (1-based) from node loads."""
    dof_loaded = {}

    for node, (Fx, Fy) in nodes_loaded.items():
        ux_dof, uy_dof = node_dofs_1based(node)
        if Fx != 0.0:
            dof_loaded[ux_dof] = dof_loaded.get(ux_dof, 0.0) + Fx
        if Fy != 0.0:
            dof_loaded[uy_dof] = dof_loaded.get(uy_dof, 0.0) + Fy

    return dof_loaded


In [57]:
def node_dofs_1based(node_id):
    """Return engineering DOF numbers (1-based): [ux_dof, uy_dof]."""
    return [2 * node_id - 1, 2 * node_id]

## Element Level

In [58]:
def element_csL(xy_i, xy_j):
    xy_i = np.asarray(xy_i, dtype=float)
    xy_j = np.asarray(xy_j, dtype=float)

    dx = xy_j[0] - xy_i[0]
    dy = xy_j[1] - xy_i[1]
    L = float(np.hypot(dx, dy))

    c = dx / L
    s = dy / L
    return float(c), float(s), float(L)

In [59]:
def build_elements_csl(elements, nodes):
    """Return dict mapping element_id -> (c, s, L)."""
    elements_csl = {}

    for e_id, (i, j, E_e, A_e) in elements.items():
        c, s, L = element_csL(nodes[i], nodes[j])
        elements_csl[e_id] = (c, s, L)

    return elements_csl

## Global Stiffness and Force

In [60]:
def initialize_global_stiffness(nodes):
    """Return zero-initialized global stiffness matrix."""
    ndof_total = 2 * len(nodes)
    return np.zeros((ndof_total, ndof_total), dtype=float)

In [61]:
def truss_element_kg(E, A, c, s, L):
    """Return 4x4 global stiffness matrix for a 2D truss element."""
    factor = (E * A) / L

    c2 = c * c
    s2 = s * s
    cs = c * s

    kg = factor * np.array(
        [
            [ c2,  cs, -c2, -cs],
            [ cs,  s2, -cs, -s2],
            [-c2, -cs,  c2,  cs],
            [-cs, -s2,  cs,  s2],
        ],
        dtype=float,
    )
    return kg

In [62]:
def element_dof_map_1based(i_node, j_node):
    """Return the 4 global DOF indices (1-based) for element (i, j).

    Order matches the 4x4 element stiffness matrix:
    [u_ix, u_iy, u_jx, u_jy]
    """
    # Engineering DOF numbers (1-based)
    dofs_i_1 = [2 * i_node - 1, 2 * i_node]
    dofs_j_1 = [2 * j_node - 1, 2 * j_node]
    dofs_1based = dofs_i_1 + dofs_j_1
    return dofs_1based

In [63]:
def assemble_global_stiffness(elements, nodes, elements_csl, print_toggle):
    """Assemble and return the global stiffness matrix K (dense) for a 2D truss."""

    K = initialize_global_stiffness(nodes)

    for e_id, (i, j, E_e, A_e) in elements.items():
        c, s, L = elements_csl[e_id]
        ke = truss_element_kg(E_e, A_e, c, s, L)
        
        dof_map = element_dof_map_1based(i, j)

        # Scatter-add ke into K. Must be 0-based indexing, hence -1
        for a in range(4):
            A = dof_map[a] - 1
            for b in range(4):
                B = dof_map[b] - 1
                K[A, B] += ke[a, b]

        # Optional: show progress while learning/debugging
        if print_toggle:
            print_matrix_scaled(ke)
            print(f"Assembled element {e_id}: nodes ({i},{j}) -> DOFs {dof_map}")
            print_matrix_scaled(K)
            print("-" * 70)

    return K

In [64]:
def assemble_global_force_vector(nodes, dof_loaded_1based):
    """Return global force vector assembled from 1-based DOF loads."""
    ndof_total = 2 * len(nodes)
    f_global = np.zeros(ndof_total, dtype=float)

    for dof_1based, value in dof_loaded_1based.items():
        f_global[dof_1based - 1] = value  # convert to 0-based index

    return f_global

## Partition

In [65]:
def partition_system(K, f, dof_restrained_1based):
    ndof = K.shape[0]

    # Convert restrained DOFs to 0-based
    restrained_dofs = sorted(d - 1 for d in dof_restrained_1based)

    # Free DOFs
    free_dofs = [i for i in range(ndof) if i not in restrained_dofs]

    # Partition stiffness matrix
    K_ff = K[np.ix_(free_dofs, free_dofs)]
    K_fr = K[np.ix_(free_dofs, restrained_dofs)]
    K_rf = K[np.ix_(restrained_dofs, free_dofs)]
    K_rr = K[np.ix_(restrained_dofs, restrained_dofs)]

    # Partition force vector
    f_f = f[free_dofs]
    f_r = f[restrained_dofs]

    return K_ff, K_fr, K_rf, K_rr, f_f, f_r, free_dofs, restrained_dofs


## Solve Displacements

In [66]:
def solve_free_displacements(K_ff, K_fr, f_f, u_r=None):
    if u_r is None:
        u_r = np.zeros(K_fr.shape[1])

    rhs = f_f - K_fr @ u_r
    u_f = np.linalg.solve(K_ff, rhs)

    return u_f


## Solve Support Forces

In [67]:
def solve_support_forces(K_rf, K_rr, u_f, u_r=None):
    if u_r is None:
        u_r = np.zeros(K_rr.shape[0])

    F_r = K_rf @ u_f + K_rr @ u_r
    return F_r


## Backward Pass

In [68]:
def assemble_global_displacements(u_f, free_dofs, restrained_dofs, u_r=None):
    """
    Assemble the full global displacement vector u from partitioned results.
    """
    ndof_total = len(free_dofs) + len(restrained_dofs)
    u_global = np.zeros(ndof_total)

    if u_r is None:
        u_r = np.zeros(len(restrained_dofs))

    u_global[free_dofs] = u_f
    u_global[restrained_dofs] = u_r

    return u_global


In [69]:
def assemble_global_forces(f_f, F_r, free_dofs, restrained_dofs):
    """
    Assemble the full global force vector f from applied loads and reactions.
    """
    ndof_total = len(free_dofs) + len(restrained_dofs)
    f_global = np.zeros(ndof_total)

    f_global[free_dofs] = f_f
    f_global[restrained_dofs] = F_r

    return f_global


In [70]:
def extract_element_displacements(u_global, i_node, j_node):
    """
    Extract the 4x1 element global displacement vector u_e.
    Order: [u_ix, u_iy, u_jx, u_jy]
    """
    dofs_1based = [2 * i_node - 1, 2 * i_node,
                   2 * j_node - 1, 2 * j_node]
    idx = [d - 1 for d in dofs_1based]  # convert to 0-based
    return u_global[idx]

In [71]:
def truss_transformation_matrix(c, s):
    return np.array(
        [
            [ c,  s,  0,  0],
            [-s,  c,  0,  0],
            [ 0,  0,  c,  s],
            [ 0,  0, -s,  c],
        ],
        dtype=float,
    )

def compute_local_displacements(c, s, u_e):
    T = truss_transformation_matrix(c, s)
    return T @ u_e



In [72]:
def truss_local_stiffness(E, A, L):
    return (E * A / L) * np.array(
        [
            [ 1, 0, -1, 0],
            [ 0, 0,  0, 0],
            [-1, 0,  1, 0],
            [ 0, 0,  0, 0],
        ],
        dtype=float,
    )

In [73]:
def compute_local_end_forces(E, A, L, u_local):
    k_local = truss_local_stiffness(E, A, L)
    return k_local @ u_local

In [74]:
def compute_axial_force_and_stress(E, A, L, u_local):
    # axial displacements are the local DOFs along the member axis
    u_i_axial = u_local[0]
    u_j_axial = u_local[2]

    N = (E * A / L) * (u_j_axial - u_i_axial)
    sigma = N / A
    return N, sigma

In [75]:
def local_to_global_forces(c, s, f_local):
    T = truss_transformation_matrix(c, s)
    return T.T @ f_local

## Post-Processing & Output

In [76]:
def plot_truss_deformation(nodes, elements, u_global, scale=1.0):
    """
    Plot original (black) and deformed (red) truss geometry.
    """
    plt.figure()

    for e_id, (i, j, *_ ) in elements.items():
        xi, yi = nodes[i]
        xj, yj = nodes[j]

        ui = u_global[2*(i-1):2*(i-1)+2]
        uj = u_global[2*(j-1):2*(j-1)+2]

        # original
        plt.plot([xi, xj], [yi, yj], "k-", lw=2)

        # deformed
        plt.plot(
            [xi + scale*ui[0], xj + scale*uj[0]],
            [yi + scale*ui[1], yj + scale*uj[1]],
            "r-", lw=2
        )

    plt.axis("equal")
    plt.xlabel("x")
    plt.ylabel("y")
    plt.title(f"Original (black) and deformed (red), scale={scale}")
    plt.show()


In [77]:
def recover_element_results(elements, elements_csl, u_global):
    results = {}

    for e_id, (i, j, E_e, A_e) in elements.items():
        c, s, L = elements_csl[e_id]

        u_e = extract_element_displacements(u_global, i, j)
        u_local = compute_local_displacements(c, s, u_e)
        f_local = compute_local_end_forces(E_e, A_e, L, u_local)
        N, sigma = compute_axial_force_and_stress(E_e, A_e, L, u_local)

        results[e_id] = {
            "u_e": u_e,
            "u_local": u_local,
            "f_local": f_local,
            "N": N,          # kN
            "sigma": sigma,  # GPa
        }

    return results

In [78]:
def build_element_results_dataframe(elements, elements_csl, results):
    """Return pandas DataFrame of element-level results."""
    rows = []

    for e_id in sorted(elements.keys()):
        i, j, _, _ = elements[e_id]
        _, _, L = elements_csl[e_id]
        r = results[e_id]

        row = {
            "ele": e_id,
            "i": i,
            "j": j,
            "L (mm)": round(L, 1),
            "N (kN)": round(r["N"], 1),
            "sigma (MPa)": round(r["sigma"] * 1000, 1),
        }

        # global displacements u_e
        row.update({f"u_{k+1} (mm)": round(r["u_e"][k], 1) for k in range(4)})
        # local displacements u'
        row.update({f"u_{k+1}' (mm)": round(r["u_local"][k], 1) for k in range(4)})
        # local end forces f'
        row.update({f"f_{k+1}' (kN)": round(r["f_local"][k], 1) for k in range(4)})

        rows.append(row)

    return pd.DataFrame(rows)


In [79]:
def display_compact(df):
    return (
        df.style
        .format({
            col: "{:.1f}"
            for col in df.columns
            if any(key in col for key in ["(mm)", "(kN)", "(MPa)", "L"])
        })
        .set_properties(**{
            "font-size": "9pt",
            "padding": "2px",
            "white-space": "nowrap",
        })
        .set_table_styles([
            {"selector": "th", "props": [("font-size", "9pt")]},
        ])
    )

In [80]:
def print_matrix_scaled(K, scale=1, decimals=2, col_width=7):
    """
    Print K/scale row-by-row, compact, with DOF labels.
    """
    fmt = f"{{:{col_width}.{decimals}f}}"
    print(f"K = {scale:.0e} ×")
    for i, row in enumerate(K, start=1):
        row_scaled = row / scale
        row_str = " ".join(fmt.format(val) for val in row_scaled)
        print(f"{i:2d} | {row_str}")

---
---

## Question 3 — Extending the Truss Analysis Code to 3D (20 points)

Extend the functions defined at the beginning of this workbook so that they operate in **three dimensions**. This requires:

- Updating the number of degrees of freedom per node from **2 → 3**
- Modifying all related indexing, assembly, and bookkeeping to be consistent with 3D kinematics
- Use the 3D local/global truss element formulation presented in class
- Applying an appropriate set of **support restraints** for a 3D truss system (including out-of-plane constraints)

Verify that the **statically determinate truss** analyzed in the written assignment produces **consistent and correct results** when solved using your 3D computational implementation. You should get the same answer across 2D and 3D implementation if you are doing things correctly.

### Implementation Guidance

As an example, in the function below, extending the formulation to 3D requires changing the total number of degrees of freedom from `2 * len(nodes)` to `3 * len(nodes)`. Similar changes will be required throughout your code so that it correctly assembles, solves, and visualizes a **3D structure**. Use a **3D axis** when plotting with `matplotlib`.

```python
def assemble_global_force_vector(nodes, dof_loaded_1based):
    """Return global force vector assembled from 1-based DOF loads."""
    ndof_total = 2 * len(nodes)
    f_global = np.zeros(ndof_total, dtype=float)

    for dof_1based, value in dof_loaded_1based.items():
        f_global[dof_1based - 1] = value  # convert to 0-based index

    return f_global


### Q3.1. Problem Setup
Define all model inputs (nodes, elements, supports, loads) using the same data structures and conventions as we did in lecture but extended to 3D.

Present:
- Node coordinates
- Element connectivity and properties
- Applied loads and boundary conditions

In [81]:
# -------------------------
# Nodes: (x,y,z)
# -------------------------
nodes = {}

In [82]:
E = None
A = None

# -------------------------
# Elements: (i, j, E, A)
# -------------------------
elements = {}

# elements_csl = build_elements_csl(elements, nodes)

In [83]:
# -------------------------
# Supports / restraints
# -------------------------
nodes_restrained = {}

# -------------------------
# Applied nodal loads (Fx, Fy, Fz)
# -------------------------
nodes_loaded = {}

dof_restrained_1based = restrained_dofs_1based(nodes_restrained, node_dofs_1based)
dof_loaded_1based = loaded_dofs_1based(nodes_loaded, node_dofs_1based)


### Q3.2 Analysis

Solve the truss using the space truss functions you need to develop (edit the code at the start of this workbook).

Report key numerical results, such as:
- Support reactions
- Member axial forces
- Maximum nodal displacements
- Present summary tables
- Plot displaced shape at a reasonable scale

In [84]:
print_toggle = 0
K_global = assemble_global_stiffness(elements, nodes, elements_csl, print_toggle)
f_global = assemble_global_force_vector(nodes, dof_loaded_1based)

(
    K_ff,
    K_fr,
    K_rf,
    K_rr,
    f_f,
    f_r,
    free_dofs,
    restrained_dofs,
) = partition_system(K_global, f_global, dof_restrained_1based)

u_f = solve_free_displacements(K_ff, K_fr, f_f)
F_r = solve_support_forces(K_rf, K_rr, u_f)
u_global = assemble_global_displacements(u_f, free_dofs, restrained_dofs)
f_global_complete = assemble_global_forces(f_f, F_r, free_dofs, restrained_dofs)


NameError: name 'elements_csl' is not defined

In [ ]:
plot_truss_deformation(nodes, elements, u_global, scale=100)

In [ ]:
results = recover_element_results(elements, elements_csl, u_global)
df_members = build_element_results_dataframe(elements, elements_csl, results)
display_compact(df_members)

In [ ]:
# Choose one element to sanity-check
e_test = 1

# --- Geometry + properties ---
i, j, E, A = elements[e_test]
c, s, L = elements_csl.get(e_test, element_csL(nodes[i], nodes[j]))

print(f"Element {e_test}: nodes ({i}, {j})")
print(f"c = {c:.6f}, s = {s:.6f}, L = {L:.3f} mm")

# --- Displacements ---
u_e = extract_element_displacements(u_global, i, j)
print(f"u_e (ele {e_test} global displacements) = {u_e}")

u_local = compute_local_displacements(c, s, u_e)
print(f"u_local (ele {e_test} local displacements) = {u_local}")

# --- Forces / stress ---
f_local = compute_local_end_forces(E, A, L, u_local)
print(f"f_local (ele {e_test} local end forces) = {f_local}")

N, sigma = compute_axial_force_and_stress(E, A, L, u_local)
print(f"Axial force  N = {N:.3f} kN")
print(f"Axial stress σ = {sigma:.6f} GPa  ({sigma*1000:.2f} MPa)")

f_global_e = local_to_global_forces(c, s, f_local)
print(f"f_global_e (ele {e_test} end forces in global coords) = {f_global_e}")


### YOUR ANSWER FOR Q3.2 IN THIS CELL:

Use as many cells as you need

---

### Q3.3 Discussion

Compare the support reactions and member foces obtained from your code with those calculated by hand and with the 2D analysis performed in Q1. State whether they agree, and briefly summarize your findings in the final cell using **Markdown** and **LaTeX** as appropriate.

### YOUR ANSWER FOR Q1.3 IN THIS CELL:

Use as many cells as you need

---
---

## Question 4 — Case Study Space Truss Analysis (30 points)

Analyze the following space truss given your 3D code that you have verified gives correct answers from Q3. 

Pin-jointed **3D space truss** with **translational DOFs only** at each node:
$$(U_x, U_y, U_z)$$

Information given in units 
- Length: m
- Force: kN

### Model Summary

| Quantity | Value |
|--------:|------:|
| Nodes | 18 |
| Members | 53 |
| Total DOFs | $18 \times 3 = 54$ |
| Restrained DOFs | 6 |
| Free DOFs | 48 |
| Structural Type | 3D space truss |
| Load Case | Vertical + lateral |

$ E = 200 $ GPa

>Careful of units, make sure to convert to same basis


### Nodes

Bottom Layer ($z = 0$)

| Node | $x$ | $y$ | $z$ |
|-----:|----:|----:|----:|
| 1 | 0 | 0 | 0 |
| 2 | 1 | 0 | 0 |
| 3 | 2 | 0 | 0 |
| 4 | 0 | 1 | 0 |
| 5 | 1 | 1 | 0 |
| 6 | 2 | 1 | 0 |
| 7 | 0 | 2 | 0 |
| 8 | 1 | 2 | 0 |
| 9 | 2 | 2 | 0 |

Top Layer ($z = 1.2$)

| Node | $x$ | $y$ | $z$ |
|-----:|----:|----:|----:|
| 10 | 0 | 0 | 1.2 |
| 11 | 1 | 0 | 1.2 |
| 12 | 2 | 0 | 1.2 |
| 13 | 0 | 1 | 1.2 |
| 14 | 1 | 1 | 1.2 |
| 15 | 2 | 1 | 1.2 |
| 16 | 0 | 2 | 1.2 |
| 17 | 1 | 2 | 1.2 |
| 18 | 2 | 2 | 1.2 |


### Members (Connectivity)

Bottom Chord – Edges

| Member | $i$ | $j$ | $A$ ($mm^2$) |
|-------:|----:|----:|-------------:|
| 1 | 1 | 2 | 1200 |
| 2 | 2 | 3 | 1200 |
| 3 | 4 | 5 | 1200 |
| 4 | 5 | 6 | 1200 |
| 5 | 7 | 8 | 1200 |
| 6 | 8 | 9 | 1200 |
| 7 | 1 | 4 | 1200 |
| 8 | 4 | 7 | 1200 |
| 9 | 2 | 5 | 1200 |
| 10 | 5 | 8 | 1200 |
| 11 | 3 | 6 | 1200 |
| 12 | 6 | 9 | 1200 |

Bottom Chord – Diagonals

| Member | $i$ | $j$ | $A$ ($mm^2$) |
|-------:|----:|----:|-------------:|
| 13 | 1 | 5 | 900 |
| 14 | 2 | 6 | 900 |
| 15 | 4 | 8 | 900 |
| 16 | 5 | 9 | 900 |

Top Chord – Edges


| Member | $i$ | $j$ | $A$ ($mm^2$) |
|-------:|----:|----:|-------------:|
| 17 | 10 | 11 | 900 |
| 18 | 11 | 12 | 900 |
| 19 | 13 | 14 | 900 |
| 20 | 14 | 15 | 900 |
| 21 | 16 | 17 | 900 |
| 22 | 17 | 18 | 900 |
| 23 | 10 | 13 | 900 |
| 24 | 13 | 16 | 900 |
| 25 | 11 | 14 | 900 |
| 26 | 14 | 17 | 900 |
| 27 | 12 | 15 | 900 |
| 28 | 15 | 18 | 900 |

Top Chord – Diagonals

| Member | $i$ | $j$ | $A$ ($mm^2$) |
|-------:|----:|----:|-------------:|
| 29 | 10 | 14 | 700 |
| 30 | 11 | 15 | 700 |
| 31 | 13 | 17 | 700 |
| 32 | 14 | 18 | 700 |

Web Members - Verticals

| Member | $i$ | $j$ | $A$ ($mm^2$) |
|-------:|----:|----:|-------------:|
| 33 | 1 | 10 | 700 |
| 34 | 2 | 11 | 700 |
| 35 | 3 | 12 | 700 |
| 36 | 4 | 13 | 700 |
| 37 | 5 | 14 | 700 |
| 38 | 6 | 15 | 700 |
| 39 | 7 | 16 | 700 |
| 40 | 8 | 17 | 700 |
| 41 | 9 | 18 | 700 |


Web Members - Cross-Layer Diagonals

| Member | $i$ | $j$ | $A$ ($mm^2$) |
|-------:|----:|----:|-------------:|
| 42 | 10 | 2 | 600 |
| 43 | 11 | 3 | 600 |
| 44 | 13 | 5 | 600 |
| 45 | 14 | 6 | 600 |
| 46 | 16 | 8 | 600 |
| 47 | 17 | 9 | 600 |
| 48 | 10 | 4 | 600 |
| 49 | 13 | 7 | 600 |
| 50 | 11 | 5 | 600 |
| 51 | 14 | 8 | 600 |
| 52 | 12 | 6 | 600 |
| 53 | 15 | 9 | 600 |



### Supports / Fixities

Translational restraints only for truss.

| Node | $U_x$ | $U_y$ | $U_z$ |
|-----:|:----:|:----:|:----:|
| 1 | fixed | fixed | fixed |
| 3 | free | fixed | fixed |
| 7 | free | free | fixed |

Total restrained DOFs: $3 + 2 + 1 = 6$

### Applied Loads

Nodal Loads (kN)

| Node | $F_x$ | $F_y$ | $F_z$ |
|-----:|-----:|-----:|-----:|
| 10 | $+2$ | $0$ | $-10$ |
| 11 | $+2$ | $0$ | $-10$ |
| 12 | $+2$ | $0$ | $-10$ |
| 13 | $+2$ | $0$ | $-10$ |
| 14 | $+2$ | $0$ | $-30$ |
| 15 | $+2$ | $0$ | $-10$ |
| 16 | $+2$ | $0$ | $-10$ |
| 17 | $+2$ | $0$ | $-10$ |
| 18 | $+2$ | $0$ | $-10$ |

All other nodes: $F_x = F_y = F_z = 0$


---

### Q4.1. Problem Setup
Define all model inputs (nodes, elements, supports, loads) using the same data structures and conventions as in **Question 3**.

Present:
- Node coordinates
- Element connectivity and properties
- Applied loads and boundary conditions

In [ ]:
# -------------------------
# Nodes: (x,y,z)
# -------------------------
nodes = {}

In [ ]:
E = None
A = None

# -------------------------
# Elements: (i, j, E, A)
# -------------------------
elements = {}

# elements_csl = build_elements_csl(elements, nodes)

In [ ]:
# -------------------------
# Supports / restraints
# -------------------------
nodes_restrained = {}

# -------------------------
# Applied nodal loads (Fx, Fy, Fz)
# -------------------------
nodes_loaded = {}

dof_restrained_1based = restrained_dofs_1based(nodes_restrained, node_dofs_1based)
dof_loaded_1based = loaded_dofs_1based(nodes_loaded, node_dofs_1based)

In [ ]:
# Plotting function for 3D trusses (Plotly version).
# This mirrors the Matplotlib, but works interactively everywhere (Jupyter, Colab, VS Code, etc.).

def plot_truss_3d_plotly(
    nodes,
    elements,
    nodes_restrained=None,
    nodes_loaded=None,
    show_node_ids=True,
    show_member_ids=False,
    load_scale=0.02,
):
    """
    nodes: {nid: (x,y,z)}
    elements: {eid: (i,j,E,A)}
    nodes_restrained: {nid: ["ux","uy","uz"], ...}
    nodes_loaded: {nid: (Fx,Fy,Fz)}
    """

    # ---- normalize nodes to 3D ----
    nodes3 = {}
    for nid, xyz in nodes.items():
        if len(xyz) == 2:
            x, y = xyz
            z = 0.0
        else:
            x, y, z = xyz
        nodes3[nid] = (float(x), float(y), float(z))

    fig = go.Figure()

    # =========================================================
    # Members (lines)
    # =========================================================
    xs, ys, zs = [], [], []
    for eid, (i, j, E, A) in elements.items():
        xi, yi, zi = nodes3[i]
        xj, yj, zj = nodes3[j]
        xs += [xi, xj, None]
        ys += [yi, yj, None]
        zs += [zi, zj, None]

    fig.add_trace(
        go.Scatter3d(
            x=xs,
            y=ys,
            z=zs,
            mode="lines",
            line=dict(width=3),
            name="Members",
        )
    )

    # Optional member IDs (text at midpoints)
    if show_member_ids:
        mx, my, mz, mt = [], [], [], []
        for eid, (i, j, E, A) in elements.items():
            xi, yi, zi = nodes3[i]
            xj, yj, zj = nodes3[j]
            mx.append((xi + xj) / 2)
            my.append((yi + yj) / 2)
            mz.append((zi + zj) / 2)
            mt.append(str(eid))

        fig.add_trace(
            go.Scatter3d(
                x=mx,
                y=my,
                z=mz,
                mode="text",
                text=mt,
                name="Member IDs",
            )
        )

    # =========================================================
    # Nodes
    # =========================================================
    nx = [p[0] for p in nodes3.values()]
    ny = [p[1] for p in nodes3.values()]
    nz = [p[2] for p in nodes3.values()]

    fig.add_trace(
        go.Scatter3d(
            x=nx,
            y=ny,
            z=nz,
            mode="markers",
            marker=dict(size=4),
            name="Nodes",
        )
    )

    if show_node_ids:
        fig.add_trace(
            go.Scatter3d(
                x=nx,
                y=ny,
                z=nz,
                mode="text",
                text=[str(nid) for nid in nodes3.keys()],
                textposition="top center",
                name="Node IDs",
            )
        )

    # =========================================================
    # Supports
    # =========================================================
    if nodes_restrained:
        sx, sy, sz = [], [], []
        for nid in nodes_restrained.keys():
            x, y, z = nodes3[nid]
            sx.append(x)
            sy.append(y)
            sz.append(z)

        fig.add_trace(
            go.Scatter3d(
                x=sx,
                y=sy,
                z=sz,
                mode="markers",
                marker=dict(size=7),
                name="Supports",
            )
        )

    # =========================================================
    # Loads (true arrows using cones)
    # =========================================================
    if nodes_loaded:
        lx, ly, lz = [], [], []
        u, v, w = [], [], []

        for nid, (Fx, Fy, Fz) in nodes_loaded.items():
            x, y, z = nodes3[nid]
            lx.append(x)
            ly.append(y)
            lz.append(z)

            # direction + magnitude
            u.append(load_scale * Fx)
            v.append(load_scale * Fy)
            w.append(load_scale * Fz)

        fig.add_trace(
            go.Cone(
                x=lx,
                y=ly,
                z=lz,
                u=u,
                v=v,
                w=w,
                anchor="tail",   # arrow starts at node
                sizemode="absolute",
                sizeref=0.2,     # adjust arrowhead size
                showscale=False,
                name="Loads",
            )
        )


    # =========================================================
    # Layout (mirrors Matplotlib intent)
    # =========================================================
    fig.update_layout(
        scene=dict(
            xaxis_title="X",
            yaxis_title="Y",
            zaxis_title="Z",
            aspectmode="data",  # equal-ish aspect
            camera=dict(
                eye=dict(x=1.4, y=1.4, z=0.9)  # similar to elev=20, azim=30
            ),
        ),
        margin=dict(l=0, r=0, t=30, b=0),
        title="3D Truss",
        showlegend=True,
    )

    fig.show()


In [ ]:
plot_truss_3d_plotly(
    nodes,
    elements,
    nodes_restrained,
    nodes_loaded,
    show_node_ids=True,
    show_member_ids=False,
    load_scale=0.015,
)


---

### Q4.2. Analysis

Solve the truss using the space truss functions you need to developed in Q3.

Report key numerical results, such as:
- Support reactions
- Member axial forces
- Maximum nodal displacements
- Present summary tables
- Plot displaced shape at a reasonable scale

In [ ]:
print_toggle = 0
K_global = assemble_global_stiffness(elements, nodes, elements_csl, print_toggle)
f_global = assemble_global_force_vector(nodes, dof_loaded_1based)

(
    K_ff,
    K_fr,
    K_rf,
    K_rr,
    f_f,
    f_r,
    free_dofs,
    restrained_dofs,
) = partition_system(K_global, f_global, dof_restrained_1based)

u_f = solve_free_displacements(K_ff, K_fr, f_f)
F_r = solve_support_forces(K_rf, K_rr, u_f)
u_global = assemble_global_displacements(u_f, free_dofs, restrained_dofs)
f_global_complete = assemble_global_forces(f_f, F_r, free_dofs, restrained_dofs)


In [ ]:
plot_truss_deformation(nodes, elements, u_global, scale=100)

In [ ]:
results = recover_element_results(elements, elements_csl, u_global)
df_members = build_element_results_dataframe(elements, elements_csl, results)
display_compact(df_members)

In [ ]:
# Choose one element to sanity-check
e_test = 1

# --- Geometry + properties ---
i, j, E, A = elements[e_test]
c, s, L = elements_csl.get(e_test, element_csL(nodes[i], nodes[j]))

print(f"Element {e_test}: nodes ({i}, {j})")
print(f"c = {c:.6f}, s = {s:.6f}, L = {L:.3f} mm")

# --- Displacements ---
u_e = extract_element_displacements(u_global, i, j)
print(f"u_e (ele {e_test} global displacements) = {u_e}")

u_local = compute_local_displacements(c, s, u_e)
print(f"u_local (ele {e_test} local displacements) = {u_local}")

# --- Forces / stress ---
f_local = compute_local_end_forces(E, A, L, u_local)
print(f"f_local (ele {e_test} local end forces) = {f_local}")

N, sigma = compute_axial_force_and_stress(E, A, L, u_local)
print(f"Axial force  N = {N:.3f} kN")
print(f"Axial stress σ = {sigma:.6f} GPa  ({sigma*1000:.2f} MPa)")

f_global_e = local_to_global_forces(c, s, f_local)
print(f"f_global_e (ele {e_test} end forces in global coords) = {f_global_e}")


### YOUR ANSWER FOR Q4.2 IN THIS CELL:

Use as many cells as you need

---


### Q4.3. Discussion
Discuss the computed **member forces** and **nodal displacements**.

Interpret your results from an engineering perspective by addressing the following questions:

- Are the computed displacements and deformed shape physically reasonable for the structure and loading considered?
- Do the signs and magnitudes of member forces and support reactions make sense?
- Which members carry the largest axial forces, and how does this relate to the expected load paths?
- How do your modeling assumptions (geometry, cross-sections, boundary conditions) influence the results?
- Which assumptions do you expect the results to be most sensitive to, and why?


### YOUR ANSWER FOR Q4.3 IN THIS CELL:

Use as many cells as you need

---

## Reflection (Required)

In 3–6 sentences:

- What part felt easiest? Describe in speciics
- What part felt hardest? Describe in speciics
- One bug you encountered and how you fixed it.
- One thing you still don’t understand.
- If you used collaboration or AI tools, briefly describe how.


### YOUR RESPONSE HERE